In [1]:
%pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Wikipedia Tool - provided by LangChain

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [3]:
## Creating a custom tool

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ssl, httpx, os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

import ssl, httpx
ssl_context = ssl.create_default_context()
ssl_context.load_default_certs()
httpx_client = httpx.Client(verify=ssl_context)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_base="https://llm-proxy-api.ai.openeng.netapp.com",
    openai_api_key=openai_api_key,
    model_kwargs={'user': 'pranitm'},
    http_client=httpx_client
)

loader = WebBaseLoader("https://blog.langchain.dev/launching-langgraph-templates/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents, embeddings)
retriever = vector_db.as_retriever()
retriever


USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002209B503770>)

In [6]:
## Create Retriever Tool - provided by LangChain

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search",
    "Search for infomation about LangSmith. For any questions regarding LangSmith, you must use this tool.")
retriever_tool.name

'langsmith_search'

In [7]:
## Arxiv Tool - provided by LangChain

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
## Combining all the tools

tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\pranitm\\Dummy\\Learning_GenAI\\LangChain\\myenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for infomation about LangSmith. For any questions regarding LangSmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000220826018A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vect

### AGENTS

In [11]:
from dotenv import load_dotenv  
load_dotenv()

import os, ssl, httpx
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
ssl_context = ssl.create_default_context()
ssl_context.load_default_certs()
httpx_client = httpx.Client(verify=ssl_context)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-35-turbo",
    openai_api_base="https://llm-proxy-api.ai.openeng.netapp.com",
    model_kwargs={'user': 'pranitm'},
    http_client=httpx_client,
    temperature=0
)

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


c:\Users\pranitm\Dummy\Learning_GenAI\LangChain\myenv\Lib\site-packages\langsmith\client.py:5431: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
## Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [14]:
## Agent Executer

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_h

In [15]:
agent_executor.invoke({"input":"Tell me about Langsmith."})

{'input': 'Tell me about Langsmith.',
 'output': 'Langsmith is a company that has developed LangGraph, a powerful language model and knowledge graph platform. LangGraph allows users to build and deploy natural language processing applications, such as chatbots and question answering systems, with ease. LangGraph Studio is the main tool provided by Langsmith for creating and managing language models and knowledge graphs. The platform also offers templates for common use cases, making it easier for developers to get started with building their applications. Langsmith is focused on providing high-quality templates and expanding the number of supported providers over time.'}

In [16]:
agent_executor.invoke({"input":"Tell me about Machine Learning."})

{'input': 'Tell me about Machine Learning.',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence that focuses on the development and study of statistical algorithms that can learn from data and generate predictions or make decisions without being explicitly programmed. In other words, machine learning algorithms are designed to automatically learn and improve from experience.\n\nThe goal of machine learning is to enable computers to learn and adapt to new information and patterns, without being explicitly programmed for every possible scenario. This is achieved by training the machine learning models on large datasets, allowing them to identify patterns, relationships, and trends in the data. Once trained, these models can be used to make predictions or decisions on new, unseen data.\n\nThere are several types of machine learning algorithms, including supervised learning, unsupervised learning, and reinforcement learning. In supervised learning, the algorit